## Deteccion del tablero en la imagen
<pre>
Se usara el tablero B para esta seccion.
</pre>
### Objetivo:
<pre>
    * Se posee una armazon de metal que permite mantener una camara del microcontrolador ESP-32 CAM por encima de un tablero de 
    ajedrez. Este armazon consta de un circulo de metal como base. Lo que se desea es poder poner un tablero con una determinada
     posicion de piezas y que mediante una foto se pueda representar computacionalmente la situacion.
    
</pre>
-----------------------------------------
### Representaciones:
<pre>
Se usa una representacion de tablero mediante un array de 64 elementos de 8 bits cada uno. En cada uno de esos elementos se
 almacenara una pieza. Aqui se presentan los encodings para cada una de ellas. 
    * Los primeros 4 bits se usaran para representar el color.
        - 0000 XXXX : Negro
        - 1111 XXXX : Blanco
    * Con los 4 bits restantes se representa la pieza:
         Pieza        Codificacion
        - Peon      | 0000
        - Caballo   | 0001
        - Alfil     | 0010
        - Torre     | 0011
        - Reina     | 0100
        - Rey       | 0101
    * Un casillero vacio se representa mediante: 1111 1111
</pre>
----------------------------------------
### Detalle del algoritmo:
<pre>
El algoritmo consta de 4 partes:
        
    1. Deteccion del tablero: Cuando la foto se captura el tablero puede estar torcido y puesto en cualquier tipo de posicion. Por 
    lo que lo primero es identificarlo, recortarlo, centrarlo y alinearlo.

    2. Luego se reconocen cada una de las casillas individuales. Se determinan sus dimensiones, mediante 2 puntos, la esquina 
    superior izquierda y la esquina inferior izquierda.

    3. Se aplica 2 mascaras. Una de ellas detecta piezas blancas, la otra negras. Si se detecta una pieza se almacena en un array de 
    64 elementos booleanos.

    4. Se compara el array de booleanos con el de piezas y se realizan las debidas actualizaciones.  

A tener en cuenta: 

Para el correcto funcionamiento del algoritmo se debe conocer la posicion inicial de las piezas, ya que en base a estas se realiza 
la deduccion de posiciones despues.
</pre>

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Cargo las imagenes del tablero b
b_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial.jpeg")
b_pos_inicio2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/pos-inicial-2.jpeg")
mov1 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-1.jpg")
mov2= cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-2.jpg")
mov3 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-3.jpg")
mov4 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-4.jpg")
mov5 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-5.jpg")
mov6 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-b/mov-6.jpg")

In [3]:
# Funciones comunes 
def showImg(img, text):
    cv2.namedWindow(text, cv2.WINDOW_KEEPRATIO)
    cv2.imshow(text, img)
    cv2.resizeWindow(text, 700, 700)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def mostrarContornos(img,min,max, bool):
    # Aplicar deteccion de bordes utilizando Canny
    edges = cv2.Canny(img, min, max, apertureSize=3)
    if bool:
        showImg(edges, 'Detector de contornos')
    return edges

# 1. Deteccion y recorte del tablero

In [4]:
# Trabajaremos con esta posicion.
showImg(b_pos_inicio, "Posicion inicial")

#### Rotamos y recortamos el tablero

In [5]:
def rotarRecortarImagen(img):
    # Convertimos la imagen a gris
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Aplicar deteccion de bordes utilizando Canny
    edges = mostrarContornos(img_gray, 100, 300, False)

    # Encontrar las lineas presentes en la imagen utilizando la transformada de Hough
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Calcular el angulo promedio solo de las líneas horizontales detectadas
    angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines if abs(line[0][3] - line[0][1]) < abs(line[0][2] - line[0][0])])

    # Rotar la imagen utilizando el ángulo calculado
    (h, w) = img_gray.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle * 180 / np.pi, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    #  Convierto la imagen a hsv
    hsv_image = cv2.cvtColor(rotated, cv2.COLOR_BGR2HSV)

    # Defino el rango de colores para poder recortar el tablero, en este caso se usa teniendo en cuenta el fondo de color blanco
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([255, 90, 200])

    # Creo una mascara que identifique aproximadamente el color #778078, este es el color del fondo.
    mask = cv2.inRange(hsv_image, lower_color, upper_color)

    # Se encuentran los contornos
    contorno,_ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cont = 0

    booleano = True

    for c in contorno:
        area = cv2.contourArea(c)
        if (area > 40000):
            cont = cont + 1
            x,y,w,h = cv2.boundingRect(c)
            # Recortamos el contorno encontrado.
            if (booleano):
                cropped_image = rotated[y:(y+h), x:(x+w)].copy()
                cropped_image = cropped_image[60:843, 55:830]
                booleano = False
            cv2.rectangle(rotated,(x,y),(x+w,y+h),(0,255,255),2)
            cv2.putText(rotated,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)
            area = cv2.contourArea(c)
    return cropped_image

### Tests

In [6]:
cropped_image = rotarRecortarImagen(b_pos_inicio) # no funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(b_pos_inicio2) # no funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov1) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov2) # no funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov3) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov4) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov5) # funciona
showImg(cropped_image, 'cropped_image')
cropped_image = rotarRecortarImagen(mov6) # funciona
showImg(cropped_image, 'cropped_image')


In [45]:
def dibujarContornos(img, masked_img, title, bool):
    retorno = img.copy()
    edges = cv2.Canny(masked_img, 200, 500).copy()
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    chessboard_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if (area > 1000):
            chessboard_contours.append(contour)
    for contour in chessboard_contours:
        cv2.drawContours(retorno , [contour], -1, (0, 255, 0), 2).copy()
    if bool:
        showImg(retorno , title)
    return retorno 


image = rotarRecortarImagen(mov1) 

# Mascara
#lower_black = np.array([0, 0, 0])  # Rango inferior para el color negro
#upper_black = np.array([80, 80, 70])  # Rango superior para el color negro
lower_black = np.array([0, 0, 0])
upper_black = np.array([50, 50, 50])
#lower_black = np.array([0,50,50],np.uint8)
#upper_black = np.array([180,255,255],np.uint8)

hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

black_mask = cv2.inRange(image, lower_black, upper_black)
black_mask_hsv = cv2.inRange(hsv, lower_black, upper_black)

#dibujarContornos(image, image, 'hsv', True)
#dibujarContornos(image, hsv, 'hsv', True)
#dibujarContornos(image, gray, 'gray', True)
#dibujarContornos(image, black_mask, 'black mask', True)
#dibujarContornos(image, black_mask_hsv, 'black mask and hsv', True)

contours, _ = cv2.findContours(black_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

grid_contours = []
for contour in contours:
    if cv2.contourArea(contour) > 900:
        grid_contours.append(contour)
    

cv2.drawContours(image, grid_contours, -1, (0, 255, 0), 2)


#contours, _ = cv2.findContours(black_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#
#grid_contours = []
#for contour in contours:
#    if cv2.contourArea(contour) > 900:
#        grid_contours.append(contour)
    

#cv2.drawContours(image, grid_contours, -1, (0, 255, 0), 2)

# hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# h, s, v = cv2.split(hsv_image)
# heatmap = cv2.applyColorMap(v, cv2.COLORMAP_HSV)
# 
# contours, _ = cv2.findContours(heatmap, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# 
# grid_contours = []
# for contour in contours:
#     if cv2.contourArea(contour) > 1000:
#         grid_contours.append(contour)


showImg(image, 'black_mask')

In [8]:
image = rotarRecortarImagen(mov1) 

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply a Gaussian blur to the image
blur_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Apply Canny edge detection to the image
edge_image = cv2.Canny(blur_image, 50, 150)

# Find the contours in the image
contours, _ = cv2.findContours(edge_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Filter the contours to find the grid
grid_contours = []
for contour in contours:
    if cv2.contourArea(contour) > 1000:
        grid_contours.append(contour)

# Draw the grid on the image
cv2.drawContours(image, grid_contours, -1, (0, 255, 0), 2)

# Display the image
showImg(image,'Grid')



In [9]:
# recorte de casilleros 

def detectarPieza(casilla1):
    blancobajo = np.array([36,100,0],np.uint8)
    blancoalto = np.array([100,255,255],np.uint8)

    mascara1 = cv2.inRange(casilla1, blancobajo, blancoalto)

    hsv = cv2.cvtColor(casilla1, cv2.COLOR_BGR2GRAY)
    #showImg(hsv, 'mascara')
    contorno,_ = cv2.findContours(hsv,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cont = 0

    for c in contorno:
        area = cv2.contourArea(c)
        cont = cont + 1
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(hsv,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.putText(hsv,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)

    #showImg(hsv, 'mascara')
    if len(contorno) > 0:
        print("Se ha detectado una pieza.")
    else:
        print("No se ha detectado una pieza.")
    return casilla1
    
x = np.array([0, 80, 180, 280, 380, 480, 580, 680, 773])
y = np.array([0, 98, 195, 285, 385, 485, 585, 685, 781])

image = rotarRecortarImagen(mov1) 
for i in range(2):
    for j in range(2):
        start_x = int(x[i])
        end_x = int(x[i]+85)
        start_y = int(y[j])
        end_y = int(y[j]+85)

        # Crop the image
        cropped_image = image[start_y:end_y, start_x:end_x]
        cropped_image = detectarPieza(cropped_image)
        showImg(cropped_image, 'cropped_image')
        
        

Se ha detectado una pieza.
Se ha detectado una pieza.
Se ha detectado una pieza.
Se ha detectado una pieza.


In [10]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv_image)
heatmap = cv2.applyColorMap(v, cv2.COLORMAP_HSV)

showImg(heatmap, 'heatmap')

-----------------------
# Tablero B

In [11]:
# Carga las imagenes en una variable
a_pos_inicio = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio.jpeg")
pos_mid = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid.jpeg")
pos_mid2 = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2.jpeg")
pos_ataque = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-ataque.jpeg")
iluminada = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/iluminada.jpeg")
pos_inicial_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-inicio-esp.jpeg")
pos_mid_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid-esp.jpeg")
pos_mid2_esp = cv2.imread("/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-a/pos-mid2-esp.jpeg")
tablero_normal = cv2.imread('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-ideal/tablero-comun.png')

In [12]:
# procesamos la imagen 
cv2.imshow('Pos mid', pos_mid)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Convert the image to grayscale
pos_mid_gray = cv2.cvtColor(pos_mid, cv2.COLOR_BGR2GRAY)

cv2.imshow('Pos mid grayscale', pos_mid_gray)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# Find the chessboard corners
cv2.imshow('tablero normal: ',tablero_normal)
cv2.waitKey(10000)
cv2.destroyAllWindows()

ret, corners = cv2.findChessboardCorners(pos_mid_gray, (9, 9))

print(ret, corners)

# Draw the chessboard corners on the image
if ret:
  cv2.drawChessboardCorners(pos_mid, (9, 9), corners, ret)

# Display the image
cv2.imshow('edges detected', pos_mid) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

False None


In [13]:
## Imagen pos mid esp32
tablero = cv2.imread('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-comun.jpeg')

# procesamos la imagen 
cv2.imshow('Pos mid ESP32', tablero)
cv2.waitKey(5000)
cv2.destroyAllWindows()

pos_mid_gray = cv2.cvtColor(tablero, cv2.COLOR_BGR2GRAY)

cv2.imshow('Pos mid grayscale', pos_mid_gray)
cv2.waitKey(5000)
cv2.destroyAllWindows()

ret, corners = cv2.findChessboardCorners(pos_mid_gray, (7,7), cv2.CALIB_CB_ADAPTIVE_THRESH)
print(ret, corners)

if ret:
  cv2.drawChessboardCorners(pos_mid_gray, (7, 7), corners, ret)

cv2.imshow('edges detected', pos_mid_gray) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

[ WARN:0@15.038] global loadsave.cpp:244 findDecoder imread_('/home/tom/universidad/LIDI/cv-tablero/tableros-img/tablero-comun.jpeg'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [ ]:
## Version 2

nline = 7
ncol = 7

## termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

## processing
gray = cv2.cvtColor(pos_mid_esp, cv2.COLOR_BGR2GRAY)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (nline, ncol), None)
corners2 = cv2.cornerSubPix(gray, corners, (7, 7), (-1, -1), criteria)


In [ ]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

# Posicion inicial

In [ ]:
# Muestra la imagen en una ventana
plt.imshow(mid_iluminada2)
plt.show()

In [ ]:
# Indica las coordenadas del área a recortar en píxeles
x1, y1 = 425, 95  # Esquina superior izquierda
x2, y2 = 1300, 990  # Esquina inferior derecha

# Recorta la imagen en base a las coordenadas indicadas
imagen_recortada = mid_iluminada2[y1:y2, x1:x2].copy()

# Muestra la imagen recortada en una ventana
plt.imshow(imagen_recortada)
plt.show()

gray = cv2.cvtColor(imagen_recortada, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)

dst = cv2.calcHist(gray, [0], None, [256], [0,256])

plt.hist(dst.ravel(),256,[0,256])
plt.title('Histogram for gray scale image')
plt.show()
#imghsv = cv2.cvtColor(imagen_recortada,cv2.COLOR_BGR2HSV)

#blancobajo = np.array([160,50,50],np.uint8)
#blancoalto = np.array([180,255,255],np.uint8)
#mascara1 = cv2.inRange(imghsv, blancobajo, blancoalto)

#cv2_imshow(mascara1)
#mascara1 = ~mascara1
#cv2_imshow(mascara1)

In [ ]:
x1, y1 = 3, 0  # Esquina superior izquierda
x2, y2 = 100, 105  # Esquina inferior derecha
casilla = imagen_recortada[y1:y2, x1:x2].copy()

plt.imshow(casilla)
plt.show()

imghsv = cv2.cvtColor(casilla,cv2.COLOR_BGR2HSV)

amarillobajo = np.array([15,50,50],np.uint8)
amarilloalto = np.array([25,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv,amarillobajo, amarilloalto)

verdebajo = np.array([36,50,50],np.uint8)
verdealto = np.array([75,255,255],np.uint8)
mascara2 = cv2.inRange(imghsv,verdebajo,verdealto)
cv2_imshow(mascara1)



In [ ]:
x1, y1 = 148, 10  # Esquina superior izquierda
x2, y2 = 270, 138  # Esquina inferior derecha
casilla = imagen_recortada[y1:y2, x1:x2].copy()


plt.imshow(casilla)
plt.show()

imghsv = cv2.cvtColor(casilla,cv2.COLOR_BGR2HSV)

amarillobajo = np.array([15,50,50],np.uint8)
amarilloalto = np.array([25,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv,amarillobajo, amarilloalto)

verdebajo = np.array([36,50,50],np.uint8)
verdealto = np.array([75,255,255],np.uint8)
mascara2 = cv2.inRange(imghsv,verdebajo,verdealto)
cv2_imshow(mascara1)



In [ ]:
x1, y1 = 410, 20  # Esquina superior izquierda
x2, y2 = 510, 128   # Esquina inferior derecha
casilla = imagen_recortada[y1:y2, x1:x2].copy()

imghsv = cv2.cvtColor(casilla,cv2.COLOR_BGR2HSV)

blancobajo = np.array([15,50,50],np.uint8)
blancoalto = np.array([25,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv,blancobajo, blancoalto)
mascara1 = ~mascara1

plt.imshow(mascara1, cmap="gray")
plt.show()

contorno,_ = cv2.findContours(mascara1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cont = 0

for c in contorno:
    area = cv2.contourArea(c)
    if (area > 60):
        cont = cont + 1
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(casilla,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.putText(casilla,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)

cv2_imshow(casilla)

if len(contorno) > 0:
    print("Se ha detectado una pieza.")
else:
    print("No se ha detectado una pieza.")

plt.imshow(mascara1)
plt.show()


In [ ]:
x1, y1 = 415, 150  # Esquina superior izquierda
x2, y2 = 515, 255  # Esquina inferior derecha
casilla1 = imagen_recortada[y1:y2, x1:x2].copy()

imghsv = cv2.cvtColor(casilla1,cv2.COLOR_BGR2HSV)

blancobajo = np.array([160,50,50],np.uint8)
blancoalto = np.array([180,255,255],np.uint8)
mascara1 = cv2.inRange(imghsv, blancobajo, blancoalto)

mascara1 = ~mascara1

contorno,_ = cv2.findContours(mascara1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cont = 0
for c in contorno:
    area = cv2.contourArea(c)
    if (area > 50):
        cont = cont + 1
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(casilla1,(x,y),(x+w,y+h),(0,255,255),2)
        cv2.putText(casilla1,str(cont),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)

cv2_imshow(casilla1)

if len(contorno) > 0:
    print("Se ha detectado una pieza.")
else:
    print("No se ha detectado una pieza.")

plt.imshow(casilla1)
plt.show()

# Posicion de middle game


# procesamiento-tablero-v2

In [ ]:
import numpy as np 
import cv2
import sys
import os


nfila = 8
ncol = 8

img = cv2.imread('/content/pos-inicial.jpg')

# El algoritmo se detiene despues de 30 iteraciones o cuando el error es de menos de 0.001
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Convierto la imagen a gris
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2_imshow(gray)

# Devuelve ret que es un booleano que determina si se encontro un tablero y cornes es una 
# lista de puntos en la imagen que representan los vertices 
ret, corners = cv2.findChessboardCorners(gray, (nfila,ncol), None)

print(corners)


# Refina las coordenadas del tablero,
if ret == True:
  corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)


